In [50]:
import captum
import torch
import thermostat
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from captum.attr import IntegratedGradients, FeatureAblation

import json

In [32]:
from datasets import load_dataset
dataset = load_dataset('imdb', ignore_verifications=True)

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /home/tim/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [44]:
ds_sorted = sorted([(i, x) for i,x in enumerate(dataset["test"])], key=lambda x: len(x[1]["text"]))

In [45]:
ds_sorted[20]

(17327,
 {'text': 'Don\'t waste your time and money on it. It\'s not quite as bad as "Adrenalin", by the same director but that\'s not saying much.',
  'label': 0})

keys should be {text, input_ids, tokens, attributions, idx)

In [96]:
tokenizer = AutoTokenizer.from_pretrained("textattack/distilbert-base-uncased-imdb")

model = AutoModelForSequenceClassification.from_pretrained("textattack/distilbert-base-uncased-imdb", return_dict=False)


In [ ]:
forward_func = lambda *x: model(*x)[0]
method = FeatureAblation(forward_func)

In [134]:
samples = []
for idx, data in ds_sorted:
    input1 = tokenizer(data["text"])
    input_ids = torch.tensor([input1["input_ids"]]).long()
    attention_mask = torch.tensor([input1["attention_mask"]]).long()
    target_class = data["label"]
    attributions, approximation_error = method.attribute((input_ids, attention_mask),
                                                 target=target_class,
                                                 method='gausslegendre',
                                                 return_convergence_delta=True)
    
    sample={"idx": idx,
            "text": data["text"],
            "attributions": attributions[0].tolist()}
    tokenized = tokenizer(data["text"])
    sample["input_ids"] = tokenized["input_ids"]
    sample["attention_mask"] = tokenized["attention_mask"]
    sample["tokens"] = tokenizer.convert_ids_to_tokens(tokenized["input_ids"])
    
    samples.append(sample)

In [139]:
with open("../data/albert-imdb-feature-ablation.json", "w") as fp:
    json.dump(samples, fp)